In [ ]:
#pip install trino urllib3 pandas

In [ ]:
# two possible apis to generate a trino connection:
import trino
# from pyhive import presto

# pandas dfs
import pandas as pd

import urllib3
urllib3.disable_warnings()

In [ ]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [ ]:
JWT_TOKEN = os.environ['TRINO_PASSWD']
conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=int(os.environ['TRINO_PORT']),
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(JWT_TOKEN),
    verify=True,
)
cur = conn.cursor()

In [ ]:
cur.execute('show catalogs')
cur.fetchall()

In [ ]:
# Show available schemas to ensure trino connection is set correctly
cur.execute('show schemas in osc_datacommons_dev')
cur.fetchall()

In [ ]:
cur.execute('show tables from osc_datacommons_dev.wri_gppd')
cur.fetchall()

In [ ]:
# CNAME is Catalog Name (osc_datacommons_dev)
# SNAME is Schema Name (wri_gppd)
# TNAME is Table Name (plants)

def trino_to_df(cname, sname, tname):
    trino_table = '.'.join([cname, sname, tname])
    cur.execute('show columns from ' + trino_table)
    columns_df = pd.DataFrame(cur.fetchall()).dropna(axis=1,how='all')
    
    cur.execute('select * from ' + trino_table)
    df = pd.DataFrame(cur.fetchall())
    df.columns = columns_df.iloc[:, 0]
    return df

In [ ]:
df = trino_to_df('osc_datacommons_dev', 'wri_gppd', 'plants')
display(df.info(verbose=True))
df.head()